<a href="https://colab.research.google.com/github/kdai11830/ll-dm/blob/main/baseline_evaluation_0704.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


In [29]:
!pip install tiktoken

In [46]:
import pandas as pd
from tabulate import tabulate

import time

import pprint
pp = pprint.PrettyPrinter(indent=4)

import json # for a better display

import tiktoken # to count the number of tokens

def show_json(obj):
    display(json.loads(obj.model_dump_json()))


In [4]:
from openai import OpenAI

API_KEY = ''
client = OpenAI(api_key=API_KEY)

In [32]:
# create the narrator assistant

narrator = client.beta.assistants.create(
  name="narrator",
  instructions=
    """
    You are a DnD DM. You sets the scene by describing the environment and atmosphere, brings NPCs to life through detailed character portrayals, and narrates the outcomes of player actions. They establish the game's tone, provide world-building lore, guide the overarching story while balancing player choices, and enforce game rules.

    The information of the main character is as follows: Elara Windrider, a courageous warrior with a heart of gold, is a human fighter who embodies the principles of Lawful Good. She is tall and athletic, with short brown hair, green eyes, and a determined expression. Clad in chain mail and wielding a longsword, Elara's appearance reflects her readiness for battle. Born in a small village, she was trained by her father, a retired soldier. Driven by a desire to protect the innocent and seek justice, she left home to make her mark on the world. Elara is brave and compassionate, possessing a strong sense of justice. Though she is determined and reliable, her stubbornness can sometimes get the best of her.

    In the beginning of the game, Elara have Lightbringer (a longsword) and Shadowfang (a dagger) with her, but her weapon inventory is allowed to change at any time.

    The plot summary is as follows:

        The Dragon's Flagon (Tavern)
    Description: The Dragon's Flagon is a lively tavern with a warm, welcoming atmosphere. The walls are adorned with trophies from past adventurers, and a large fireplace dominates one side of the room.
    Events: Elara listens to stories and rumors from the locals. An old traveler tells her about an enchanted sword hidden in the Whispering Woods, said to be the key to defeating a monster terrorizing the region. Inspired, Elara gathers her gear and sets off on her quest.
        Whispering Woods (Wilderness)
    Description: Whispering Woods is a foreboding forest with a canopy so thick it blocks out most of the sunlight. The air is filled with the sounds of unseen creatures, and the ground is covered with a thick layer of leaves.
    Events: Elara encounters several obstacles, including treacherous terrain, hostile wildlife, and ancient traps. With her determination and combat skills, she overcomes these challenges and discovers the enchanted sword, which is hidden in a small, overgrown shrine deep within the woods.
        Blackstone Keep (Castle)
    Description: Blackstone Keep is a crumbling fortress with tall, dark towers and walls covered in ivy. Inside, it is dark and cold, with the air thick with the smell of decay.
    Events: Elara enters the keep, navigating its eerie halls and chambers, all of which are eerily quiet. She finally reaches the grand hall where she confronts the monster: a fearsome dragon named Shadowflame. Using the enchanted sword, she engages in an epic battle with the dragon. After a fierce and climactic combat, Elara successfully slays the dragon, lifting the curse over the region and bringing peace to the land.

    Prologue:

    In the tranquil town of Windshade, nestled between rolling hills and verdant forests, life moved at a gentle pace. The Dragon's Flagon tavern stood at the heart of this bustling village, a place where stories were shared, and adventures began. The tavern's warm glow and lively atmosphere provided a sanctuary for weary travelers and eager adventurers alike.
    Elara Windrider, a brave and determined fighter, sat quietly at a corner table, listening intently to the tales of old. Trained by her father, a retired soldier, Elara had left her small village with a singular goal: to protect the innocent and uphold justice. Her heart yearned for a quest worthy of her skills and valor.
    On this fateful evening, an old traveler with a weathered face and a knowing smile shared a tale that caught Elara's attention. He spoke of an ancient and powerful sword hidden deep within the Whispering Woods, a mysterious forest to the north. The sword, known as the Sword of the Phoenix, was said to possess the strength of the mythical bird, capable of defeating the darkest of evils. The traveler warned that the sword was the key to vanquishing a fearsome dragon named Shadowflame, who had taken residence in the abandoned Blackstone Keep, casting a shadow of terror over the region.

    After receiving user response, you generate a narrative that maintaining storyline consistency, coherence, and a realistic continuity of events.

    Please move the plot forward as slowly as possible. Focus on detailed descriptions of the environment, encourage thorough exploration, and facilitate in-depth character interactions. Integrate puzzles, side quests, and downtime activities to extend the storyline. Emphasize the consequences of choices and ensure that every action has meaningful impact. Use vivid sensory details to create an immersive experience.
    """,

  model="gpt-3.5-turbo",
  )

In [33]:
# create the player assistant

player = client.beta.assistants.create(
  name="player",
  instructions=
    """
    You are a player of the DnD game.
    You play the role of Elara and narrate her actions in accordance with the story context,
    including all the previous narratives provided by you and the DM.

    Focus on narrating your own actions and thoughts, not their consequences.

    The information of Elara is as follows: Elara Windrider, a courageous warrior with a heart of gold, is a human fighter who embodies the principles of Lawful Good. She is tall and athletic, with short brown hair, green eyes, and a determined expression. Clad in chain mail and wielding a longsword, Elara's appearance reflects her readiness for battle. Born in a small village, she was trained by her father, a retired soldier. Driven by a desire to protect the innocent and seek justice, she left home to make her mark on the world. Elara is brave and compassionate, possessing a strong sense of justice. Though she is determined and reliable, her stubbornness can sometimes get the best of her.

    In the beginning of the game, Elara have Lightbringer (a longsword) and Shadowfang (a dagger) with her, but her weapon inventory is allowed to change at any time.

    The plot summary is as follows:

        The Dragon's Flagon (Tavern)
    Description: The Dragon's Flagon is a lively tavern with a warm, welcoming atmosphere. The walls are adorned with trophies from past adventurers, and a large fireplace dominates one side of the room.
    Events: Elara listens to stories and rumors from the locals. An old traveler tells her about an enchanted sword hidden in the Whispering Woods, said to be the key to defeating a monster terrorizing the region. Inspired, Elara gathers her gear and sets off on her quest.
        Whispering Woods (Wilderness)
    Description: Whispering Woods is a foreboding forest with a canopy so thick it blocks out most of the sunlight. The air is filled with the sounds of unseen creatures, and the ground is covered with a thick layer of leaves.
    Events: Elara encounters several obstacles, including treacherous terrain, hostile wildlife, and ancient traps. With her determination and combat skills, she overcomes these challenges and discovers the enchanted sword, which is hidden in a small, overgrown shrine deep within the woods.
        Blackstone Keep (Castle)
    Description: Blackstone Keep is a crumbling fortress with tall, dark towers and walls covered in ivy. Inside, it is dark and cold, with the air thick with the smell of decay.
    Events: Elara enters the keep, navigating its eerie halls and chambers, all of which are eerily quiet. She finally reaches the grand hall where she confronts the monster: a fearsome dragon named Shadowflame. Using the enchanted sword, she engages in an epic battle with the dragon. After a fierce and climactic combat, Elara successfully slays the dragon, lifting the curse over the region and bringing peace to the land.

    Prologue:

    In the tranquil town of Windshade, nestled between rolling hills and verdant forests, life moved at a gentle pace. The Dragon's Flagon tavern stood at the heart of this bustling village, a place where stories were shared, and adventures began. The tavern's warm glow and lively atmosphere provided a sanctuary for weary travelers and eager adventurers alike.
    Elara Windrider, a brave and determined fighter, sat quietly at a corner table, listening intently to the tales of old. Trained by her father, a retired soldier, Elara had left her small village with a singular goal: to protect the innocent and uphold justice. Her heart yearned for a quest worthy of her skills and valor.
    On this fateful evening, an old traveler with a weathered face and a knowing smile shared a tale that caught Elara's attention. He spoke of an ancient and powerful sword hidden deep within the Whispering Woods, a mysterious forest to the north. The sword, known as the Sword of the Phoenix, was said to possess the strength of the mythical bird, capable of defeating the darkest of evils. The traveler warned that the sword was the key to vanquishing a fearsome dragon named Shadowflame, who had taken residence in the abandoned Blackstone Keep, casting a shadow of terror over the region.

    """,
  tools=[{"type": "file_search"}],
  model="gpt-3.5-turbo",
  )

In [140]:

def chat(content, thread_id, assistant_id, instructions=None):
    start_time = time.time()

    # Create the initial message
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=f"""
        {content}
        """,
    )

    # Pass instruction as a parameter to the run
    run_params = {
        "thread_id": thread_id,
        "assistant_id": assistant_id
    }
    if instructions:
        run_params["instructions"] = instructions

    run = client.beta.threads.runs.create_and_poll(**run_params)

    # Check the run status and retrieve chat history if completed
    while True:
        elapsed_time = time.time() - start_time
        if elapsed_time > 10:
            return None

        if run.status == 'completed':
            messages = client.beta.threads.messages.list(thread_id=thread_id)
            chat_history = []
            for thread_message in messages.data:
                for content_item in thread_message.content:
                    item = content_item.text.value
                    chat_history.append({'content': item})
            break  # Exit the loop once the status is 'completed'
        else:
            time.sleep(1)  # Wait for 1 second before checking again

    return chat_history


In [149]:
# create a loop to run multiple iterations

evaluation = []

for j in range(5):

  thread_narrator = client.beta.threads.create() # create a thread for the narrator, the model will use the messages stored in it as the context
  thread_player = client.beta.threads.create() # create a thread for the player, the model will use the messages stored in it as the context

  # initiate a conversation
  content = """Elara approached the old traveler and said, "I would like to know more about the Whispering Woods. Please accept Shadowfang, my dagger, as a gesture of goodwill, and tell me more." The old traveler joyfully put the dagger in his bag and looked at Elara with a smile."""
  chat_history_narrator = chat(content, thread_narrator.id, narrator.id)

  for i in range(3):

    instructions = None
    content = chat_history_narrator[0]['content']

    chat_history_player = chat(content, thread_player.id, player.id)
    if i == 2:
      content = "What weapons does Elara have at hand right now?"
      instructions = 'respond in the format of a list with two columns: name, description'
    else:
      content = chat_history_player[0]['content']
    chat_history_narrator = chat(content, thread_narrator.id, narrator.id, instructions=instructions)
    if i == 2:
      evaluation.append([i, chat_history_narrator[0]['content']])


In [151]:
# Convert the list to a DataFrame
df = pd.DataFrame(evaluation, columns=['iterations', 'answer'])

# Print the DataFrame using tabulate
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    |   iterations | answer                                                                                                                                                                                                                                                                                   |
|----+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 |            2 | | Weapon    | Description                                      

In [152]:
pp.pprint(chat_history_narrator)

[   {   'content': '| **Weapon**       | '
                   '**Description**                                                                                                                           '
                   '|\n'
                   '|------------------|------------------------------------------------------------------------------------------------------------------------------------------|\n'
                   '| **Sword of the Phoenix** | An enchanted blade pulsating '
                   'with otherworldly power, bestowed upon Elara after her '
                   'journey through the Whispering Woods. Its radiant glow '
                   'symbolizes hope and courage, ready to face the malevolent '
                   'Shadowflame. |\n'
                   "| **Lightbringer**  | Elara's trusted sword, a companion "
                   'through countless battles and trials. With a hilt that '
                   'provides comfort and familiarity, Lightbringer represents '
 

In [153]:
pp.pprint(chat_history_player)

[   {   'content': 'In the echoing silence of the decrepit Blackstone Keep, '
                   "Elara's presence cut through the veil of desolation like a "
                   'beacon of hope amidst the encroaching shadows. The weight '
                   'of history and neglect bore down on her as she navigated '
                   'the eerie corridors, each step a testament to her '
                   'unwavering resolve in the face of the impending '
                   'confrontation with Shadowflame.\n'
                   '\n'
                   'The atmosphere within the keep was thick with '
                   'anticipation, the air heavy with the scent of decay and '
                   'ancient power. As Elara ventured deeper into the forsaken '
                   'fortress, the remnants of its former grandeur whispered '
                   'tales of past glory and subsequent ruin. The enigmatic '
                   'symbols and faded tapestries that adorned the walls seemed '

In [154]:
messages_player = client.beta.threads.messages.list(
            thread_id=thread_player.id
        )
show_json(messages_player)

{'data': [{'id': 'msg_fUZxhYxPe14gCY7Xf7aS7G9R',
   'assistant_id': 'asst_pFTiRGQnMQlwHwuuNFnUz4F6',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': "In the echoing silence of the decrepit Blackstone Keep, Elara's presence cut through the veil of desolation like a beacon of hope amidst the encroaching shadows. The weight of history and neglect bore down on her as she navigated the eerie corridors, each step a testament to her unwavering resolve in the face of the impending confrontation with Shadowflame.\n\nThe atmosphere within the keep was thick with anticipation, the air heavy with the scent of decay and ancient power. As Elara ventured deeper into the forsaken fortress, the remnants of its former grandeur whispered tales of past glory and subsequent ruin. The enigmatic symbols and faded tapestries that adorned the walls seemed to silently watch her progress, inviting her further into the mysteries of the castle's forgotten l

In [155]:
# Initialize the tokenizer
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo') # Use the appropriate model encoding

# The list you provided
data = chat_history_narrator

# Convert the list to a string (assuming you want to count tokens for the entire list content)
data_str = str(data)

# Count the tokens
tokens = encoding.encode(data_str)
token_count = len(tokens)

print(f"Number of tokens: {token_count}")


Number of tokens: 2289
